In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from pandas import DatetimeIndex
import socket, struct 

pd.set_option('display.max_columns', 500)
%pip install folium

In [ ]:
df = pd.read_parquet('merged_simargl.gzip')

In [ ]:
print("This dataset has {} columns and {} rows".format(df.shape[1],df.shape[0]))

In [ ]:
df.info()

In [ ]:
df.describe(include=[object])

In [ ]:
df.describe()

In [ ]:
# Can ignore NaN values
df.nunique()

In [ ]:
df.duplicated().sum()

In [ ]:
print("Total de valores nulos")
print(df.isnull().sum())

In [ ]:
cols = ["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "DST_TO_SRC_SECOND_BYTES","SRC_TO_DST_SECOND_BYTES", "Attack", "L7_PROTO_NAME", "PROTOCOL_MAP"]

fig_nr_columns = 2
fig_nr_rows = 4
fig = plt.figure(1, figsize=[15,20])
percentage = 2

for k in range(len(cols)):
    value_counts = df[cols[k]].value_counts()
    percentage_values = (value_counts / len(df)) * 100
    filtered_values = percentage_values[percentage_values > percentage]
    filtered_values_others = percentage_values[percentage_values <= percentage]
    filtered_values_others_sum = filtered_values_others.sum()
    others_series = pd.Series({f'Others ({filtered_values_others_sum:.1f}%)\n': filtered_values_others_sum})
    filtered_values = pd.concat([filtered_values, others_series])
    labels = filtered_values.index
    sizes = filtered_values.values
    ax = fig.add_subplot(fig_nr_rows, fig_nr_columns, k+1)
    ax.pie(sizes, labels=None, autopct=None)
    plt.axis('equal')
    plt.title("Column: {}".format(cols[k]), fontweight='bold', fontsize=15)
    legend_labels = [f'{label}: {size:.1f}%' for label, size in zip(labels, sizes)]
    plt.legend(legend_labels, loc='best')
    count_less_than_two_percent = len(filtered_values_others)
    plt.annotate(f' Other -> Number of rows with less than {percentage} percent: {count_less_than_two_percent}', xy=(0.5, 0.0), xycoords='axes fraction',ha='center', va='center')
    
plt.show()

In [ ]:
cols = ["L4_SRC_PORT","L4_DST_PORT","PROTOCOL","IN_BYTES","IN_PKTS","OUT_BYTES","OUT_PKTS","TCP_FLAGS","FLOW_DURATION_MILLISECONDS",
        "MIN_IP_PKT_LEN","MAX_IP_PKT_LEN", "SRC_TO_DST_SECOND_BYTES", "DST_TO_SRC_SECOND_BYTES", "RETRANSMITTED_IN_BYTES","RETRANSMITTED_IN_PKTS","RETRANSMITTED_OUT_BYTES","RETRANSMITTED_OUT_PKTS",
        "TCP_WIN_MAX_IN","TCP_WIN_MAX_OUT", "Label", "PROTOCOL"]

fig_nr_columns = 2
fig_nr_rows = 17
fig = plt.figure(1, figsize=[40,70])

for k in range(len(cols)):
    num_bins = 100
    ax = fig.add_subplot(fig_nr_rows, fig_nr_columns, k+1)
    ax.hist(df[cols[k]], num_bins)
    plt.title("Column: {}".format(cols[k]), fontweight='bold')
    plt.ylabel("Count")

plt.show()

In [ ]:
def is_outlier(points, thresh=3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh


cols = ["L4_SRC_PORT","L4_DST_PORT","PROTOCOL","IN_BYTES","IN_PKTS","OUT_BYTES","OUT_PKTS","TCP_FLAGS","FLOW_DURATION_MILLISECONDS",
        "MIN_IP_PKT_LEN","MAX_IP_PKT_LEN", "SRC_TO_DST_SECOND_BYTES", "DST_TO_SRC_SECOND_BYTES", "RETRANSMITTED_IN_BYTES","RETRANSMITTED_IN_PKTS","RETRANSMITTED_OUT_BYTES","RETRANSMITTED_OUT_PKTS",
        "TCP_WIN_MAX_IN","TCP_WIN_MAX_OUT", "Label", "PROTOCOL"]

fig_nr_columns = 2
fig_nr_rows = 12
c = 0
fig = plt.figure(1, figsize=[25,37])
for k in range(0,24,2):
    num_bins = 100
    plt.subplots_adjust(hspace=0.3, wspace=0.1)
    ax = fig.add_subplot(fig_nr_rows, fig_nr_columns, k+1)
    yvalues = np.array(df[cols[c]].dropna())
    ax.boxplot(yvalues, vert=False)
    plt.title("{} Raw  [{} - {}]".format(cols[c], yvalues.min(), yvalues.max()), fontweight='bold')
    
    ax = fig.add_subplot(fig_nr_rows, fig_nr_columns, k+2)
    yvalues_outliers = yvalues[~is_outlier(yvalues)]
    percenteOutliers = ( 1 - ( len(yvalues_outliers)/len(yvalues) ) ) * 100
    ax.boxplot(yvalues_outliers, vert=False)
    plt.title("{} Without Outliers [{} - {}] - {}%".format(cols[c], yvalues_outliers.min(), yvalues_outliers.max(),round(percenteOutliers,2)), fontweight='bold')
    c += 1
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(30, 15))
sns.scatterplot(data=df, x="L4_SRC_PORT", y="L4_DST_PORT", hue="Attack", ax=ax)